In [27]:
import numpy as np
import faiss
import matplotlib.pyplot as plt
exec(open("../database.py").read())
exec(open("../helperFunctions.py").read())
import pandas as pd
from keras.datasets import mnist
import math
from time import perf_counter

***Prepare dataset***

In [28]:
name = 'gist-960-euclidean'
datasetTrainImages, datasetTestImages, _ = get_ann_benchmark_data(name)

Dataset gist-960-euclidean is not cached; downloading now ...


<string>:57: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


trainDataset :  (1000000, 960)
testDataset :  (1000, 960)


<string>:58: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
<string>:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.


In [29]:
datasetTestImages

array([[0.0117, 0.0115, 0.0087, ..., 0.0124, 0.0603, 0.0866],
       [0.0395, 0.0384, 0.0593, ..., 0.0386, 0.0403, 0.0222],
       [0.0152, 0.0244, 0.0535, ..., 0.0047, 0.007 , 0.0078],
       ...,
       [0.0396, 0.0456, 0.02  , ..., 0.0509, 0.0568, 0.0785],
       [0.0499, 0.0529, 0.055 , ..., 0.0433, 0.0339, 0.0639],
       [0.0254, 0.0231, 0.0288, ..., 0.0371, 0.015 , 0.017 ]],
      dtype=float32)

***Create faiss***

In [30]:
index = faiss.IndexFlatL2(datasetTrainImages.shape[1])   # take basic index
print('index.is_trained : ', index.is_trained)

index.add(datasetTrainImages) # add vectors to the index
print('index.ntotal : ', index.ntotal)

index.is_trained :  True
index.ntotal :  1000000


In [31]:
D = []
I = []
def createTrueLabels(par, D, I, datasetTestImages):
    k = 100
    totalTime = 0
    for i in range(par) : 
        xq = datasetTestImages[i:i+1] # Use the first image as the query vector
        time_start = perf_counter()
        x, y = index.search(xq, k)
        time_end = perf_counter()
        totalTime += (time_end - time_start)
        D.append(np.sqrt(x[0]))
        I.append(y[0])
    print(totalTime)
    
createTrueLabels(1000, D, I, datasetTestImages)

***Check if results consistent***

In [ ]:
# firstD = np.copy(D)
# firstI = np.copy(I)

In [ ]:
# for i in range(len(I)) :
#     for j in range(len(I[i])):
#         if (I[i][j] != firstI[i][j]):
#             print("Not equal")
#             print(I[i][j], firstI[i][j])
#             print(i, j)
#             break
#     break

In [ ]:
# print(I[0][:10])
# print(firstI[0][:10])

In [ ]:
print(len(I), len(I[0]))

1000 100


***ExportDatabase into excel***

In [ ]:
indexes = np.array(I)
distances = np.array(D)

In [ ]:
# draw_mnist(indexes, distances, datasetTrainImages)

In [ ]:
naming = name + '-true-labels.xlsx'
exportDB(indexes, distances, naming, 1)

indexes :  (1000, 100)
distances :  (1000, 100)
(1000, 2)
<bound method NDFrame.head of                                                Indexes  \
0    [18094, 53939, 18352, 52468, 15081, 29768, 213...   
1    [8572, 31348, 3884, 9533, 36846, 24556, 28082,...   
2    [285, 38143, 3421, 39889, 9708, 34763, 59938, ...   
3    [8903, 53024, 10359, 43266, 45767, 36567, 4371...   
4    [21043, 12634, 42157, 52774, 35790, 57696, 111...   
..                                                 ...   
995  [41654, 11994, 5241, 33109, 35115, 33690, 1384...   
996  [38378, 19113, 1945, 55820, 8711, 57565, 30013...   
997  [2462, 52739, 41222, 50319, 51646, 23742, 5899...   
998  [59812, 45497, 20966, 23632, 22343, 17090, 271...   
999  [49609, 44225, 51327, 58621, 14038, 47098, 585...   

                                             Distances  
0    [482.2966, 681.9905, 708.4991, 729.6321, 762.0...  
1    [1308.002, 1329.3134, 1382.7317, 1387.0912, 13...  
2    [466.0322, 538.5378, 555.8795, 599.7641